"""
Phase 1 — Scraping OMS & Forbes Afrique
Objectif : collecter des articles récents sur la santé (OMS) et l'innovation (Forbes Afrique).

Sortie principale :
  outputs/raw_articles_oms_forbes.csv  (colonnes: source,title,date,url,text)

Usage :
  pip install -r requirements.txt
  python phase1_scrape.py
"""

# OMS

In [1]:
#   pip install selenium webdriver-manager beautifulsoup4 pandas python-dateutil

import time
import re
import random
import os
import pandas as pd
from urllib.parse import urljoin
from dateutil import parser as dateparser

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException, StaleElementReferenceException

# webdriver-manager pour geckodriver
from webdriver_manager.firefox import GeckoDriverManager

In [2]:
import requests
from bs4 import BeautifulSoup

url = "https://www.who.int/fr/news"
html = requests.get(url)

soup = BeautifulSoup(html.text, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html lang="fr">
 <head>
  <!-- head to scrape:on -->
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   Communiqués de presse de l'Organisation mondiale de la Santé
  </title>
  <link href="/favicon.ico" rel="shortcut icon"/>
  <link href="/manifest.json" rel="manifest"/>
  <link href="/apple-touch-icon-precomposed.png" rel="apple-touch-icon"/>
  <meta content="#007eb4" name="theme-color">
   <link href="https://fonts.googleapis.com" rel="preconnect">
    <link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect">
     <link crossorigin="" href="https://use.fontawesome.com" rel="preconnect">
      <link href="https://cdnjs.cloudflare.com" rel="dns-prefetch"/>
      <link href="https://kendo.cdn.telerik.com" rel="dns-prefetch"/>
      <link href="https://kendo.cdn.telerik.com/2021.1.119/js/kendo.all.min.js" rel="prefetch"/>
      <lin

In [3]:
# -------- CONFIG ----------
BASE = "https://www.who.int"
url = "https://www.who.int/fr/news"
OUT_DIR = "../outputs"
OUT_FILE = os.path.join(OUT_DIR, "articles_oms.csv")

nb_articles = 50   # nombre total d'articles à récupérer
MAX_PAGES = 40         # sécurité : nombre max de pages à visiter
MIN_P_LEN = 30         # longueur minimale d'un <p> pour être conservé

os.makedirs(OUT_DIR, exist_ok=True)

# -------- Options Firefox ----------
options = Options()

options.headless = False  # mettre False pour debug / voir le navigateur
options.set_preference("intl.accept_languages", "fr-FR,fr")
options.set_preference("dom.webdriver.enabled", False)
options.set_preference("useAutomationExtension", False)

# ---------- Initialisations -----------
driver = webdriver.Firefox()

# stockage simple
sources, titres, dates, urls, textes = [], [], [], [], []

count = 0
page = 1

# aller à la page de listing initiale
driver.get(url)
time.sleep(1.0)

wait = WebDriverWait(driver, 15)

In [4]:
while count < nb_articles and page <= MAX_PAGES:

    print(f"\n--- Page principale {page} ---")
    # attendre éléments de listing
    try:
        wait.until(EC.any_of(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.link-container")),
            #EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[role='listitem']"))
        ))
    except TimeoutException:
        print("Timeout: aucun élément de listing trouvé sur la page.")
        break


    items = []
    # essayer d'abord <a class="link-container">
    try:
        elems = driver.find_elements(By.CSS_SELECTOR, "a.link-container")
        if elems:
            for e in elems:
                href = e.get_attribute("href")
                title = e.get_attribute("aria-label") or e.text or ""
                # date
                try:
                    ts = e.find_element(By.CSS_SELECTOR, "span.timestamp")
                    date_txt = ts.text.strip()
                except:
                    date_txt = ""
                items.append((title.strip(), date_txt, href))
    except Exception:
        elems = []

    # # fallback containers role=listitem
    # if not items:
    #     try:
    #         containers = driver.find_elements(By.CSS_SELECTOR, "div[role='listitem'], div.list-view--item, li.list-item")
    #         for c in containers:
    #             try:
    #                 a = c.find_element(By.CSS_SELECTOR, "a[href]")
    #                 href = a.get_attribute("href")
    #                 title = a.get_attribute("aria-label") or a.text or ""
    #             except:
    #                 href = None
    #                 title = ""
    #             try:
    #                 ts = c.find_element(By.CSS_SELECTOR, "span.timestamp, .date")
    #                 date_txt = ts.text.strip()
    #             except:
    #                 date_txt = ""
    #             if href:
    #                 items.append((title.strip(), date_txt, href))
    #     except Exception:
    #         pass

    # # fallback global search
    # if not items:
    #     try:
    #         a_all = driver.find_elements(By.TAG_NAME, "a")
    #         for a in a_all:
    #             href = a.get_attribute("href") or ""
    #             if "/fr/news" in href or "/news/item" in href:
    #                 title = a.get_attribute("aria-label") or a.text or ""
    #                 items.append((title.strip(), "", href))
    #     except Exception:
    #         pass


    # clean duplicates
    seen = set()
    cleaned = []
    for t, d, h in items:
        if not h or h in seen:
            continue
        seen.add(h)
        cleaned.append((t, d, h))

    print(f"{len(cleaned)} éléments trouvés après suppression des doublons.")

    # parcourir chaque item (ouvrir article)
    for title, date_listing, href in cleaned:
        if count >= nb_articles:
            break
        if not href:
            continue

        # ouvrir article
        try:
            driver.get(href)
        except Exception as e:
            print("Erreur navigation vers", href, ":", e)
            continue

        # attendre présence de <p>
        try:
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "p")))
        except TimeoutException:
            print("Temps écoulé sur l'article : ", href)
            sources.append("OMS")
            titres.append(title or "Titre non trouvé")
            try:
                date_iso = dateparser.parse(date_listing, dayfirst=True).isoformat() if date_listing else ""
            except:
                date_iso = date_listing or ""
            dates.append(date_iso)
            urls.append(href)
            textes.append("Texte non trouvé (timeout)")
            count += 1
            continue

        # récupérer <p>
        ps = driver.find_elements(By.TAG_NAME, "p")
        texts = []
        for p in ps:
            try:
                txt = p.text.strip()
            except:
                txt = ""
            if txt and len(txt) >= MIN_P_LEN:
                texts.append(txt)

        # # fallback: chercher div spécifique si aucun <p> utile
        # if not texts:
        #     try:
        #         body = driver.find_element(By.CSS_SELECTOR, "div[data-placeholder-label='Body'], div.sf-detail-body-wrapper, div.PageContent")
        #         lis = body.find_elements(By.CSS_SELECTOR, "p, li")
        #         for el in lis:
        #             txt = el.text.strip()
        #             if txt and len(txt) >= MIN_P_LEN:
        #                 texts.append(txt)
        #     except:
        #         pass

        final_text = "\n\n".join(texts) if texts else "Texte non trouvé"

        # date prioritaire: time[datetime] ou meta article:published_time
        date_iso = ""
        try:
            time_el = driver.find_element(By.TAG_NAME, "time")
            dt = time_el.get_attribute("datetime") or time_el.text
            if dt:
                try:
                    date_iso = dateparser.parse(dt, dayfirst=True).isoformat()
                except:
                    date_iso = dt
        except:
            # chercher meta
            try:
                metas = driver.find_elements(By.TAG_NAME, "meta")
                for m in metas:
                    name = m.get_attribute("property") or m.get_attribute("name") or ""
                    if name == "article:published_time":
                        dt = m.get_attribute("content")
                        try:
                            date_iso = dateparser.parse(dt, dayfirst=True).isoformat()
                        except:
                            date_iso = dt
                        break
            except:
                pass

        if not date_iso:
            try:
                date_iso = dateparser.parse(date_listing, dayfirst=True).isoformat() if date_listing else ""
            except:
                date_iso = date_listing or ""

        # stocker
        sources.append("OMS")
        titres.append(title or "Titre non trouvé")
        dates.append(date_iso)
        urls.append(href)
        textes.append(final_text)

        count += 1
        print(f"[{count}] {title[:80]}... (text length: {len(final_text)})")

        time.sleep(random.uniform(1.0, 2.0))

    # pagination : cliquer sur Next
     

    # lire le href du premier item avant le clic (pour détecter le changement)
    try:
        first_before = None
        fe = driver.find_element(By.CSS_SELECTOR, "a.link-container[href], div[role='listitem'] a[href]")
        first_before = fe.get_attribute("href")
    except Exception:
        first_before = None

    # essayer de cliquer sur le lien data-page = next (si on connaît current page)
    # tentative simple : chercher le premier lien k-pager-nav avec data-page numérique > current (ou simplement le 1er data-page)
    clicked = False
    try:
        # 1) récupérer tous les liens pager qui ont data-page
        pager_links = driver.find_elements(By.CSS_SELECTOR, "a.k-link.k-pager-nav[data-page]")
        # construire liste de (page_int, element)
        cand = []
        for el in pager_links:
            dp = el.get_attribute("data-page")
            try:
                dpn = int(dp)
                cand.append((dpn, el))
            except:
                continue
        cand = sorted(cand, key=lambda x: x[0])
        # choisir premier candidat qui n'est pas disabled (ou le second élément si le premier est 'first' ou 'prev')
        for dpn, el in cand:
            cls = el.get_attribute("class") or ""
            aria_disabled = el.get_attribute("aria-disabled") or ""
            if "k-state-disabled" in cls or aria_disabled == "true":
                continue
            # scroll et click via JS (plus fiable)
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
            driver.execute_script("arguments[0].click();", el)
            clicked = True
            print("Clicked pager data-page =", dpn)
            break

        # 2) si pas trouvé / pas cliqué : fallback jQuery click (utile pour Kendo)
        if not clicked:
            # selector plus permissif : premier lien with title next OR first pager link > 1
            try:
                # essayer selector by title/aria-label (anglais ou fr)
                sel = "a.k-link.k-pager-nav[title*='next'], a.k-link.k-pager-nav[aria-label*='next']"
                driver.execute_script("if(window.jQuery){var el=$(arguments[0]); if(el.length){el.trigger('click');}}", sel)
                # si jQuery absent, essayer document.querySelector
                driver.execute_script("""
                    var el = document.querySelector("a.k-link.k-pager-nav[title*='next'], a.k-link.k-pager-nav[aria-label*='next']");
                    if(!el){ el = document.querySelector('a.k-link.k-pager-nav[data-page]'); }
                    if(el){ el.scrollIntoView({block:'center'}); el.click(); }
                """)
                clicked = True
                print("Fallback click attempted (jQuery/document.querySelector).")
            except Exception as e:
                print("Fallback click failed:", e)

    except Exception as e:
        print("Erreur lors de la recherche/ tentative de click next:", e)
        clicked = False

    # attendre la mise à jour : comparer first_before et first_after
    if clicked:
        waited = 0.0
        changed = False
        while waited < 15.0:
            time.sleep(0.5)
            waited += 0.5
            try:
                fe2 = driver.find_element(By.CSS_SELECTOR, "a.link-container[href], div[role='listitem'] a[href]")
                first_after = fe2.get_attribute("href")
                if first_before and first_after and first_after != first_before:
                    changed = True
                    break
                # sinon vérifier si le pager input a changé (optionnel)
                try:
                    inp = driver.find_element(By.CSS_SELECTOR, "input.k-textbox[aria-label], input.k-textbox")
                    val = inp.get_attribute("value") or inp.get_attribute("aria-label") or ""
                    # si présent et différent -> page changed
                    # (si tu sais lire la page actuelle tu peux comparer)
                except:
                    pass
            except StaleElementReferenceException:
                changed = True
                break
            except Exception:
                continue
        if not changed:
            print("Attention : la page n'a pas semblé changer après le clic (timeout).")
            # tu peux retenter ou mettre headless=False pour debugger visuellement
        else:
            print("Pagination OK : page suivante chargée.")
    else:
        print("Aucun clic effectué pour Next -> fin pagination.")
        # tu peux break ici si tu veux arrêter la boucle
        break
    # ---------------------------------------------------------------------------------------
    

# fin loop
driver.quit()



--- Page principale 1 ---
20 éléments trouvés après suppression des doublons.
[1] L’OMS publie des orientations pour faire face à la baisse brutale du financement... (text length: 4320)
[2] L’OMS publie un guide mondial pour des sociétés saines, prospères et résilientes... (text length: 6549)
[3] Lancement d’un cours en ligne sur l’utilisation de l’Atlas du CIRC pour la détec... (text length: 1976)
[4] L’OMS condamne le massacre de patients et de civils dans un contexte d’escalade ... (text length: 5093)
[5] Selon un nouveau rapport du Lancet Countdown, l’inaction climatique tue des mill... (text length: 4872)
[6] Tisser la santé par l’apprentissage : l’Académie de l’OMS relance le Pacific Ope... (text length: 5383)
[7] Les Fidji sont le 26e pays à éliminer le trachome en tant que problème de santé ... (text length: 5509)
[8] Renforcer les capacités des pays à tirer parti de la santé numérique pour des sy... (text length: 2604)
[9] Lancement de la Health Works Leaders Coalition pour f

In [5]:
# sauvegarde
df = pd.DataFrame({
    "source": sources,
    "title": titres,
    "date": dates,
    "url": urls,
    "text": textes
})

df.to_csv(OUT_FILE, index=False, encoding="utf-8-sig")
#df.to_excel("../outputs/articles_oms2.xlsx", index=False, engine='openpyxl')

print(f"\nDone. {len(df)} articles enregistré sur {OUT_FILE}")


Done. 20 articles enregistré sur ../outputs\articles_oms.csv
